In [8]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [9]:
np.random.seed(500)
corpus=pd.read_csv(r"https://raw.githubusercontent.com/Gunjitbedi/Text-Classification/master/corpus.csv",encoding='latin-1')

In [10]:
corpus['text'].dropna(inplace=True)
corpus['text']=[entry.lower() for entry in corpus['text']]
corpus['text']= [word_tokenize(entry) for entry in corpus['text']]
corpus['text']=[entry for entry in corpus['text'] if not entry in stopwords.words('english')]
tag_map=defaultdict(lambda : wn.NOUN)
tag_map['V']=wn.VERB
tag_map['A']=wn.ADJ
tag_map['R']=wn.ADV

In [11]:
for i,entry in enumerate(corpus['text']):
    finalset=[]
    word_lemmatization=WordNetLemmatizer()
    for word,tag in pos_tag(entry):
        if word.isalpha():
            wordset=word_lemmatization.lemmatize(word,tag_map[tag[0]])
            finalset.append(wordset)
    corpus.loc[i,'text_final']=str(finalset)

In [12]:
test_X, train_X, test_Y, train_Y =model_selection.train_test_split(corpus['text_final'],corpus['label'],test_size=0.3)
encoder=LabelEncoder()
train_Y=encoder.fit_transform(train_Y)
test_Y=encoder.fit_transform(test_Y)

In [13]:
tfidf_vect=TfidfVectorizer(max_features=5000)
tfidf_vect.fit(corpus['text_final'])
train_X_tfidf=tfidf_vect.transform(train_X)
test_X_tfidf=tfidf_vect.transform(test_X)

In [14]:
Svm=svm.SVC(C=1.0,kernel='linear',degree=3,gamma='auto')
Svm.fit(train_X_tfidf,train_Y)
predict_svm=Svm.predict(test_X_tfidf)
print("SVM Accuracy Score -> ",round(accuracy_score(predict_svm, test_Y)*100,4))

SVM Accuracy Score ->  84.0714
